# Barcodes

Show that barcodes don't matter and that we can pick out when they do.

In [ ]:
%load_ext autoreload
%autoreload 2
import sys
sys.path.append('../../src')
from util import *
from config import *
import ipynbname

### Barcodes comparison scatter plot
- x-axis = barcode1 
- y-axes = barcode2
- fit: 
    - x=y (line of identity)
    - best-fit line
    - Pearson R
- if more than 2 barcode reps then heatmap pearson R everything vs. everything 

In [ ]:
savefig2(ipynbname.path(), '[A] Barcodes comparison scatter plot')

### Combined barcode reps in sample
- x-axis = barcode1/barcode2 names 
- y-axis = pearson R 
- ranked by R
- Or is there a better way to do this–% percent total??


In [ ]:
savefig2(ipynbname.path(), '[B] Combinedbarcode reps in sample')